# Event summaries

In [ ]:
# Uncomment to enable debugging
# TEST_SLUG = 'contains-strong-language'
if 'TEST_SLUG' in vars() and TEST_SLUG is not None:
    import yaml

    slug = TEST_SLUG

    with open('./params.yaml') as f:
        params = yaml.load(f, yaml.SafeLoader)['events'][slug]

    spektrix_event_ids = params['spektrix_event_ids']
    venue_postcode = params['venue_postcode']
    rosterfy_event_ids = params['rosterfy_event_ids']
    project_ids = params['project_ids']
    venue_ids = params['venue_ids']
    evaluation_categories = params['evaluation_categories']


In [2]:
spektrix_event_ids: str
venue_postcode: str
rosterfy_event_ids: str
project_ids: str
evaluation_categories: str
venue_ids: str
slug: str

In [3]:
assert 'slug' in vars() and slug is not None, "'slug' not defined'"

In [19]:
import json
from datetime import datetime

import petl as etl 
from utils.paths import SITE
from utils.process.event import Sustainability, Tickets, Volunteers

from utils.themes.programme import Programme as _Programme
from utils.themes.programme_slice import ProgrammeSlice

In [20]:
OUTPUT_DIR = SITE / 'events' / slug / '_data/event'
PROGRAMME_DIR = OUTPUT_DIR / 'programme'
PROGRAMME_DIR.mkdir(exist_ok=True, parents=True)
TICKETS_DIR = OUTPUT_DIR / 'tickets'
TICKETS_DIR.mkdir(exist_ok=True, parents=True)
VOLUNTEERS_DIR = OUTPUT_DIR / 'volunteers'
SUSTAINABILITY_DIR = OUTPUT_DIR / 'sustainability'

## Programme processing

In [ ]:
class Programme:
    def __init__(self, project_ids: list[str], venue_ids: list[str], evaluation_categories: list[str]):
        self.project_ids = project_ids

        venues = (
            _Programme.venues
            .rename({ 'id': 'venue', 'organisation_or_venue_name': 'venue_name' })
            .cut('venue', 'venue_name')
        )

        all_events = (
            ProgrammeSlice().events_data
            .recast(samplesize=1_000_000, reducers={
                'audience': sum,
                'audience_eval': sum,
                'events': sum,
                'participants': sum,
            })
            .convert('venue', lambda f: f[0])
            .replace(['audience', 'participants'], None, 0)
            .leftjoin(venues)
        )

        # self.projects = _Programme.projects.selectin('id', project_ids)

        event_by_programme = (
            all_events
            .selectin('project_id', project_ids)
        )

        event_by_venue = (
            all_events
            .selectin('venue', venue_ids)
        )

        events_by_evaluation_category = (
            all_events
            .selectin('evaluation_category', evaluation_categories)
        )

        self.events = (
            etl.cat(
                event_by_programme,
                event_by_venue,
                events_by_evaluation_category,
            )
        )

    def summarise(self):
        counts = dict(
            zip(
                ('projects', 'events', ),
                (
                    self.events.distinct('project_name').nrows(),
                    self.events.replace('events', None, 0).values('events').sum(),
                ),
            )
        )
        categories = dict(
            self.events
            .aggregate(None, {
                'evaluation': ('evaluation_category', set),
                'programme': ('programme_category', list),
            })
            .convert('evaluation', list)
            .convert('programme', lambda l: list({e for s in l for e in s}))
            .convert(['evaluation', 'programme'], sorted)
            .transpose()
        ) if self.events.nrows() > 0 else None

        events_summary = dict(
            self.events
            # TODO PUT HERE?
            # .replace(['audience', 'participants', 'volunteers'], None, 0)
            .aggregate(None, {
                'audience': ('audience', sum),
                'participants': ('participants', sum),
                # 'volunteers': ('volunteers', sum),
                # 'volunteerShifts': ('volunteer_shifts', sum),
                'earliestDate': ('date', lambda dates: min(d for d in dates if d)),
                'latestDate': ('date', lambda dates: max(d for d in dates if d)),
            })
            .convert(['earliestDate', 'latestDate'], lambda f: f.isoformat())
            .transpose()
        ) if self.events.nrows() > 0 else None

        return {
            'count': counts,
            'categories': categories,
            'events': events_summary,
        }

In [ ]:
if project_ids or venue_ids or evaluation_categories:
    programme = Programme(project_ids=project_ids.split(), venue_ids=venue_ids.split(), evaluation_categories=[x for x in evaluation_categories.split(',') if x])

    with open(PROGRAMME_DIR / 'summary.json', 'w') as f:
        json.dump(programme.summarise(), f, indent=2)
    (
        programme.events
        .replace('events', None, 0)
        .aggregate(['venue_name', 'project_name'], {
            'start_date': ('date', min),
            'end_date': ('date', max),
            'event_count': ('events', sum),
            'audience': ('audience', sum),
            'participants': ('participants', sum),
        })
        .sort('start_date')
        .convert(['start_date', 'end_date'], lambda x: x.isoformat())
        .tojson(PROGRAMME_DIR / 'total_by_venue.json', indent=2)
    )

In [33]:
if spektrix_event_ids:
    tickets = Tickets(spektrix_event_ids.split())
    tickets.detailed().tocsv(TICKETS_DIR / 'by_geography.csv')
    tickets.summarise().tocsv(TICKETS_DIR / 'summary.csv')
    tickets.types().tocsv(TICKETS_DIR / 'by_type.csv')

    (
        tickets.all
        .selecteq('geography_type', 'oslaua')
        .aggregate('instance_id', sum, 'count_of_tickets', field='tickets')
        .join(tickets.instances)
        .cut('event_name', 'start', 'tickets')
        .sort('start')
        .tocsv(TICKETS_DIR / 'by_event.csv')
    )

In [ ]:
if rosterfy_event_ids:
    VOLUNTEERS_DIR.mkdir(exist_ok=True, parents=True)
    volunteers = Volunteers(str(rosterfy_event_ids).split())
    volunteers.summarise().tocsv(VOLUNTEERS_DIR / 'summary.csv')

In [ ]:
if project_ids:
    SUSTAINABILITY_DIR.mkdir(exist_ok=True, parents=True)
    sustainability = Sustainability(project_ids.split()).summarise()
    sustainability.tocsv(SUSTAINABILITY_DIR / 'breakdown.csv')
    sustainability.select(lambda r: r.impact_type == 'ALL' and r.scope == 'ALL').cut('tCO2e').tojson(SUSTAINABILITY_DIR / 'summary.json', lines=True)